In [1]:
%pylab inline
%config InlineBackend.figure_format = 'retina'
from ipywidgets import interact
import scipy.stats as stats
import pandas as pd
# import pymc3 as pm
import arviz as az
import seaborn as sns

import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
tfb = tfp.bijectors

import warnings # the warning spam is pointless and annoying
warnings.simplefilter(action="ignore", category=FutureWarning)

Populating the interactive namespace from numpy and matplotlib


### For this assigment, use the code provided in the cells below together with the lecture code in the Week 10 notebook

# Q1 Sampling from a biomodal target distribution
Use the MCMC samplers in the Tensorflow Probability package, specifically the Hamiltonian Monte Carlo and No U Turn Sampler, to sample from a multipmodel density in two dimensions. Throughout this problem, our target distribution will be the sum of two Gaussian densities. The terget density has two modes or peaks in the top left and bottom right corner, with the origin in between them.

Throughout this problem, use the initial condition (code provided below) for all chains, $(x_0, y_0) = (-2, 2)$



## A
Using `sigma=2.0` in the target distribution, sample 4 parallel chains each with 1000 steps. Use 100 burnin steps. To sample the target density, you will use one of the following MCMC algorithms,
  * `NoUTurnSampler`
  * `HamiltonianMonteCarlo`
  
and one step size adapter,
  * `SimpleStepSizeAdaptation`
  * `DualAveragingStepSizeAdaptation`


That gives four possible combinations of algorithm + step size adapter. For each combination compute the following
  1. plot the combined sample set along side the target density (e.g., with `pcolor` or `contourf`)
  2. print the mean of the sampled 2D positions (the exact mean is at the origin)
  3. print the acceptance ratio for each independent chain
  4. print the effective sample size for each independent chain (use `effective_sample_size`)
  
Finally, write what you think the best combination is and explain why.

## B
Repeat part A with `sigma=1.0`.

## C
Repeat part A with `sigma=0.5`.

## D
Does a large effective sample size always indicate an accurate estimate of the mean?

In [2]:
def make_target_log_density(sigma=None):
    G1 = tfd.MultivariateNormalDiag(loc=[-2., 2.], scale_diag=sigma*ones([2]))
    G2 = tfd.MultivariateNormalDiag(loc=[2., -2.], scale_diag=sigma*ones([2]))
    def target_log_density(x):
        return tf.math.log(G1.prob(x) + G2.prob(x))
    return target_log_density

target_L_part_A = make_target_log_density(sigma=2.0)
xtest = normal(0, 1, 2)
print(target_L_part_A(xtest))

tf.Tensor(-3.4451185011149517, shape=(), dtype=float64)


In [3]:
Nchains = 4

Xinit = ones((Nchains, 2))
Xinit[:, 0] = -2.